In [ ]:
!pip install accelerate -U
!pip install datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c62c0a75865f62a065fbd7513b1bfa03fe0a329a0f1e2469b236a139ad1f52f6
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import torch
import transformers, datasets, evaluate, seqeval
from datasets import load_dataset, load_metric
from transformers import AutoModelForTokenClassification, RobertaTokenizerFast
import numpy as np

In [ ]:
data_set = load_dataset('Babelscape/multinerd',split='train').select(range(1000000))

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
data_set_valid = load_dataset('Babelscape/multinerd',split='validation').select(range(100000))

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
data_set_test = load_dataset('Babelscape/multinerd',split='test')

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
en_data = data_set.filter(lambda x: x['lang']=='en' and (set(x['ner_tags']) & {1,2,3,4,5,6,7,8,13,14}))

Filter:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [ ]:
en_data_v = data_set_valid.filter(lambda x: x['lang']=='en' and (set(x['ner_tags']) & {1,2,3,4,5,6,7,8,13,14}))

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
en_data_test = data_set_test.filter(lambda x: x['lang']=='en' and (set(x['ner_tags']) & {1,2,3,4,5,6,7,8,13,14}))

Filter:   0%|          | 0/335986 [00:00<?, ? examples/s]

In [ ]:
len(en_data),len(en_data_v)

(110723, 13126)

In [ ]:
# labels2ids = {
#     "O": 0,
#     "B-PER": 1,
#     "I-PER": 2,
#     "B-ORG": 3,
#     "I-ORG": 4,
#     "B-LOC": 5,
#     "I-LOC": 6,
#     "B-ANIM": 7,
#     "I-ANIM": 8,
#     "B-BIO": 9,
#     "I-BIO": 10,
#     "B-CEL": 11,
#     "I-CEL": 12,
#     "B-DIS": 13,
#     "I-DIS": 14,
#     "B-EVE": 15,
#     "I-EVE": 16,
#     "B-FOOD": 17,
#     "I-FOOD": 18,
#     "B-INST": 19,
#     "I-INST": 20,
#     "B-MEDIA": 21,
#     "I-MEDIA": 22,
#     "B-MYTH": 23,
#     "I-MYTH": 24,
#     "B-PLANT": 25,
#     "I-PLANT": 26,
#     "B-TIME": 27,
#     "I-TIME": 28,
#     "B-VEHI": 29,
#     "I-VEHI": 30,
#   }

In [ ]:
# ids2labels = {value:key for key,value in labels2ids.items()}
# label_list = [key for key,value in labels2ids.items()]

In [ ]:
labels2ids_B = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-DIS": 9,
    "I-DIS": 10
  }

In [ ]:
ids2labels_B = {value:key for key,value in labels2ids_B.items()}
label_list_B = [key for key,value in labels2ids_B.items()]

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained('numind/entity-recognition-general-sota-v1',add_prefix_space=True,padding=True, truncation=True)

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],padding=True, truncation=True,is_split_into_words=True,return_tensors="pt")
    req_label_list = [0,1,2,3,4,5,6,7,8,13,14]

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        #print(label)
        label = [x if x in req_label_list else 0 for x in label]       # keeping only the required tags and replaceing zero for others
        label = [9 if x==13 else 10 if x==14 else x for x in label ]   # changing tags from 13,14 to 9,10 to be in serial order
        examples["ner_tags"][i] = label
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Setting the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["ner_tags"] = examples["ner_tags"]
    #print(examples)
    return tokenized_inputs

In [ ]:
def tokenize_and_align_labels_original(examples):
    tokenized_inputs = tokenizer(examples["tokens"],padding=True, truncation=True,is_split_into_words=True,return_tensors="pt")

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Setting the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
mapped_data = en_data.map(tokenize_and_align_labels,batched=True,batch_size=1)

Map:   0%|          | 0/110723 [00:00<?, ? examples/s]

In [ ]:
mapped_data_v = en_data_v.map(tokenize_and_align_labels,batched=True,batch_size=1)

Map:   0%|          | 0/13126 [00:00<?, ? examples/s]

In [ ]:
mapped_data_test = en_data_test.map(tokenize_and_align_labels,batched=True,batch_size=1)

Map:   0%|          | 0/29512 [00:00<?, ? examples/s]

In [ ]:
mapped_data.set_format(type="torch", columns=["input_ids","attention_mask"], output_all_columns=True)
mapped_data_v.set_format(type="torch", columns=["input_ids","attention_mask"], output_all_columns=True)
mapped_data_test.set_format(type="torch", columns=["input_ids","attention_mask"], output_all_columns=True)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('numind/entity-recognition-general-sota-v1',num_labels=11, id2label=ids2labels_B, label2id=labels2ids_B)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at numind/entity-recognition-general-sota-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [ [ids2labels_B[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    true_labels = [ [ids2labels_B[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
task="ner"
batch_size=35

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-19-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
# FOR TRAINING MODEL & VALIDATION

model_name = 'numind/entity-recognition-general-sota-v1'.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-X",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=mapped_data,
    eval_dataset=mapped_data_v,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.021400,0.022782,0.947207,0.962147,0.954619,0.991514


TrainOutput(global_step=3164, training_loss=0.036663355402337616, metrics={'train_runtime': 360.7428, 'train_samples_per_second': 306.931, 'train_steps_per_second': 8.771, 'total_flos': 3638468569705884.0, 'train_loss': 0.036663355402337616, 'epoch': 1.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

events.out.tfevents.1701879749.e009f643e635.3752.0:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

'https://huggingface.co/Saketh/entity-recognition-general-sota-v1-finetuned-ner-X/tree/main/'